Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 

# Datalist Generator

## Setup environment

In [50]:
!python3 -c "import monai" || pip install -q "monai"

## Setup imports

In [51]:
import os
import json
import random
import shutil
import tempfile
from monai.config import print_config
from monai.apps import download_and_extract
print_config()

MONAI version: 1.1.0
Numpy version: 1.24.1
Pytorch version: 1.13.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: a2ec3752f54bfc3b40e7952234fbeb5452ed63e3
MONAI __file__: /Users/udiram/Dropbox/Kidney_Volume_Recognition/code/monai_tutorials/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: NOT INSTALLED or UNKNOWN VERSION.
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.64.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.4
pandas version: NOT INSTALLED or UNKNOWN VERSION.
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLE

# Setup paths to your data

In [52]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/var/folders/gr/5q39rjj57fs4z15w99hz0pqh0000gn/T/tmpspufmjq8


# Download sample MSD Dataset

In [53]:
msd_task = "Task04_Hippocampus"
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/" + msd_task + ".tar"

compressed_file = os.path.join(root_dir, msd_task + ".tar")
dataroot = os.path.join(root_dir, msd_task)

if not os.path.exists(dataroot):
    download_and_extract(resource, compressed_file, root_dir)

2023-01-11 08:54:52,530 - INFO - Downloaded: /var/folders/gr/5q39rjj57fs4z15w99hz0pqh0000gn/T/tmpspufmjq8/Task04_Hippocampus.tar
2023-01-11 08:54:52,531 - INFO - Expected md5 is None, skip md5 check for file /var/folders/gr/5q39rjj57fs4z15w99hz0pqh0000gn/T/tmpspufmjq8/Task04_Hippocampus.tar.
2023-01-11 08:54:52,533 - INFO - Writing into directory: /var/folders/gr/5q39rjj57fs4z15w99hz0pqh0000gn/T/tmpspufmjq8.


# MSD dataset structure follows the following convention:

In [54]:
test_dir = os.path.join(dataroot, "imagesTs/")
train_dir = os.path.join(dataroot, "imagesTr/")
label_dir = os.path.join(dataroot, "labelsTr/")

# Construct skeleton JSON to populate with your own data

In [55]:
datalist_json = {
    "testing": [],
    "training": []
}

# Populate JSON with test data

In [56]:
for file in os.listdir(test_dir):
    datalist_json["testing"].append({
        "image": './imagesTs/' + file,
    })

# Visualise testing data

In [57]:
datalist_json['testing'][:10]

[{'image': './imagesTs/hippocampus_267.nii.gz'},
 {'image': './imagesTs/hippocampus_379.nii.gz'},
 {'image': './imagesTs/hippocampus_208.nii.gz'},
 {'image': './imagesTs/hippocampus_275.nii.gz'},
 {'image': './imagesTs/hippocampus_131.nii.gz'},
 {'image': './imagesTs/hippocampus_140.nii.gz'},
 {'image': './imagesTs/hippocampus_076.nii.gz'},
 {'image': './imagesTs/hippocampus_115.nii.gz'},
 {'image': './imagesTs/hippocampus_168.nii.gz'},
 {'image': './imagesTs/hippocampus_119.nii.gz'}]

# Populate with training images and labels in your directory

In [58]:
for file in os.listdir(train_dir):
    datalist_json["training"].append({
        "image": './imagesTr/' + file,
        "label": './labelsTr/' + file,
        "fold": 0  # Initialize as single fold
    })

# Visualise training data

In [59]:
datalist_json['training'][:10]

[{'image': './imagesTr/hippocampus_367.nii.gz',
  'label': './labelsTr/hippocampus_367.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_304.nii.gz',
  'label': './labelsTr/hippocampus_304.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_204.nii.gz',
  'label': './labelsTr/hippocampus_204.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_279.nii.gz',
  'label': './labelsTr/hippocampus_279.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_308.nii.gz',
  'label': './labelsTr/hippocampus_308.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_375.nii.gz',
  'label': './labelsTr/hippocampus_375.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_216.nii.gz',
  'label': './labelsTr/hippocampus_216.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_316.nii.gz',
  'label': './labelsTr/hippocampus_316.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_089.nii.gz',
  'label': './labelsTr/hippocampus_089.nii.gz',
  'fold': 0},
 {'image':

# Randomise training data


In [60]:
random.seed(42)
random.shuffle(datalist_json['training'])
datalist_json['training'][:10]

[{'image': './imagesTr/hippocampus_236.nii.gz',
  'label': './labelsTr/hippocampus_236.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_189.nii.gz',
  'label': './labelsTr/hippocampus_189.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_188.nii.gz',
  'label': './labelsTr/hippocampus_188.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_330.nii.gz',
  'label': './labelsTr/hippocampus_330.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_375.nii.gz',
  'label': './labelsTr/hippocampus_375.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_280.nii.gz',
  'label': './labelsTr/hippocampus_280.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_038.nii.gz',
  'label': './labelsTr/hippocampus_038.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_229.nii.gz',
  'label': './labelsTr/hippocampus_229.nii.gz',
  'fold': 0},
 {'image': './imagesTr/._hippocampus_001.nii.gz',
  'label': './labelsTr/._hippocampus_001.nii.gz',
  'fold': 0},
 {'ima

# Split training data into N random folds

In [61]:
num_folds = 5
fold_size = len(datalist_json['training']) // num_folds
for i in range(num_folds):
    for j in range(fold_size):
        datalist_json['training'][i * fold_size + j]['fold'] = i

# Visualise final training data with all randomised folds

In [62]:
datalist_json['training'][:5]

[{'image': './imagesTr/hippocampus_236.nii.gz',
  'label': './labelsTr/hippocampus_236.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_189.nii.gz',
  'label': './labelsTr/hippocampus_189.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_188.nii.gz',
  'label': './labelsTr/hippocampus_188.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_330.nii.gz',
  'label': './labelsTr/hippocampus_330.nii.gz',
  'fold': 0},
 {'image': './imagesTr/hippocampus_375.nii.gz',
  'label': './labelsTr/hippocampus_375.nii.gz',
  'fold': 0}]

# Save JSON to file

In [63]:
with open('datalist.json', 'w', encoding='utf-8') as f:
    json.dump(datalist_json, f, ensure_ascii=False, indent=4)

# Cleanup temporary files

In [64]:
if directory is None:
    shutil.rmtree(root_dir)